In [134]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from typing import List

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andrey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/andrey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [135]:
# Зчитуємо HTML-файл та витягуємо текст з тегів <p>, <h2> та <h1>
with open('kotsiubynskyy-mykhaylo-mykhaylovych-tini-zabutykh-predkiv1058.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')
    paragraphs = soup.find_all(['p', 'h1', 'h2'])
    text = '\n'.join([p.get_text() for p in paragraphs])

In [136]:
def write(tokens: List[str], filename: str) -> None:
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(tokens))
#список слів з їх частотами
def word_list_to_freq_dict(wordlist):
    res = {}
    for word in wordlist:
        if word not in res:
            res[word] = 1
        else:
            res[word] += 1
    return res

In [137]:
# Очищаємо текст від знаків пунктуації та цифр
text = re.sub(r"[^\w\s]", '', text)
text = re.sub(r"\d+", '', text)

In [138]:
# Токенізація тексту
text = word_tokenize(text)

In [139]:
# Завантажуємо список стоп-слів
nlp = spacy.load('uk_core_news_sm')
stop_words = nlp.Defaults.stop_words

In [140]:
# лематизація текстових елементів
lemmatized_words = [doc.lemma_ for doc in nlp(' '.join(text))]

In [141]:
# Виконуємо очищення тексту від стоп-слів
text = [word for word in lemmatized_words if word not in stop_words and re.match(r'\w+', word)]
write(text,"Lemmatisation.txt")

In [142]:
# Ініціалізуємо об'єкт CountVectorizer
vectorizer = CountVectorizer()
# Виконуємо векторизацію тексту
bag_of_words = vectorizer.fit_transform(text)
# Отримуємо список всіх слів у "Bag of Words"
word_list = vectorizer.get_feature_names_out()
write(word_list,"Bag of Words.text")

In [143]:
# Ініціалізуємо об'єкт TfidfVectorizer
top_words = word_list_to_freq_dict(text)
top_words_list = sorted(top_words.items(), key=lambda x: x[1], reverse=True)[:10]
top_words_only = [word for word, _ in top_words_list]
text = " ".join(text)

In [144]:
vectorizer = TfidfVectorizer(vocabulary=top_words_only)
tfidf_matrix = vectorizer.fit_transform([text])
tfidf_values = tfidf_matrix.toarray()[0]

word_tfidf = dict(zip(top_words_only, tfidf_values))

# Виводимо слова та значения метрики TF-IDF
for word, tfidf in word_tfidf.items():
    print(f"Слово: {word}, TF-IDF: {tfidf}")

Слово: іван, TF-IDF: 0.6143246159081622
Слово: гора, TF-IDF: 0.3751915439439112
Слово: свій, TF-IDF: 0.2721169439593202
Слово: ліс, TF-IDF: 0.251502023962402
Слово: марічка, TF-IDF: 0.251502023962402
Слово: око, TF-IDF: 0.24737903996301835
Слово: нога, TF-IDF: 0.24737903996301835
Слово: од, TF-IDF: 0.251502023962402
Слово: рука, TF-IDF: 0.22676411996610016
Слово: вівця, TF-IDF: 0.2102721839685656
